In [1]:
# === КЛАССИФИКАЦИЯ: SI (высокая селективность) ===

# Импорты
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, roc_curve
import pandas as pd
import numpy as np
import joblib
import os
import matplotlib.pyplot as plt

# Создаём папку для моделей
os.makedirs('models', exist_ok=True)

# Загрузка данных
df = pd.read_csv('data/cleaned_data.csv')

# Вычисление медианы SI
med_si = df['SI'].median()
print(f"📊 Медиана SI (селективный индекс): {med_si:.2f}")

# Целевая переменная: 1 — если SI > медианы (более селективное), 0 — менее селективное
y = (df['SI'] > med_si).astype(int)
print(f"🔍 Распределение классов:\n{y.value_counts(normalize=True)}")

# Загрузка признаков
from utils import load_data
X = load_data()  # только признаки

# Разделение с балансом классов
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print(f"✅ Признаков: {X.shape[1]}, Объектов: {X.shape[0]}")
print(f"🎯 Обучающая выборка: {X_train.shape[0]}, тестовая: {X_test.shape[0]}")

📊 Медиана SI (селективный индекс): 3.86
🔍 Распределение классов:
SI
1    0.5
0    0.5
Name: proportion, dtype: float64
✅ Признаков: 211, Объектов: 998
🎯 Обучающая выборка: 798, тестовая: 200


In [2]:
# === ОБУЧЕНИЕ МОДЕЛЕЙ ===

models = {
    "Logistic Regression": (
        LogisticRegression(max_iter=1000),
        {'C': [0.1, 1, 10], 'solver': ['liblinear']}
    ),
    "Random Forest": (
        RandomForestClassifier(random_state=42),
        {
            'n_estimators': [50, 100],
            'max_depth': [8, 10],
            'min_samples_split': [5, 10]
        }
    ),
    "Gradient Boosting": (
        GradientBoostingClassifier(random_state=42),
        {
            'n_estimators': [100],
            'learning_rate': [0.05, 0.1],
            'max_depth': [5]
        }
    ),
    "SVM": (
        SVC(probability=True),
        {
            'C': [1, 10],
            'kernel': ['rbf'],
            'gamma': ['scale']
        }
    )
}

results = []

for name, (model, params) in models.items():
    print(f"\n🔬 Обучение: {name}")
    
    grid = GridSearchCV(model, params, cv=5, scoring='roc_auc', n_jobs=-1)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    
    y_pred = best_model.predict(X_test)
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]
    
    auc = roc_auc_score(y_test, y_pred_proba)
    acc = accuracy_score(y_test, y_pred)
    
    results.append({
        'Model': name,
        'AUC': auc,
        'Accuracy': acc,
        'Params': grid.best_params_
    })
    
    print(f"  Лучшие параметры: {grid.best_params_}")
    print(f"  AUC: {auc:.4f}, Accuracy: {acc:.4f}")
    
    # Сохранение модели
    joblib.dump(
        best_model,
        f'models/best_si_median_model_{name.replace(" ", "_").lower()}.pkl'
    )


🔬 Обучение: Logistic Regression
  Лучшие параметры: {'C': 0.1, 'solver': 'liblinear'}
  AUC: 0.5879, Accuracy: 0.5000

🔬 Обучение: Random Forest
  Лучшие параметры: {'max_depth': 8, 'min_samples_split': 10, 'n_estimators': 50}
  AUC: 0.9997, Accuracy: 0.9800

🔬 Обучение: Gradient Boosting
  Лучшие параметры: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 100}
  AUC: 1.0000, Accuracy: 1.0000

🔬 Обучение: SVM
  Лучшие параметры: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
  AUC: 0.4122, Accuracy: 0.5000


In [3]:
# === РЕЗУЛЬТАТЫ ===

results_df = pd.DataFrame(results).sort_values('AUC', ascending=False)
print("\n" + "="*60)
print("📊 РЕЗУЛЬТАТЫ КЛАССИФИКАЦИИ: SI > медианы")
print("="*60)
print(results_df[['Model', 'AUC', 'Accuracy']])

best = results_df.iloc[0]
print(f"\n✅ ЛУЧШАЯ МОДЕЛЬ:")
print(f"   → {best['Model']} (AUC = {best['AUC']:.4f}, Accuracy = {best['Accuracy']:.4f})")
print(f"   → Параметры: {best['Params']}")


📊 РЕЗУЛЬТАТЫ КЛАССИФИКАЦИИ: SI > медианы
                 Model      AUC  Accuracy
2    Gradient Boosting  1.00000      1.00
1        Random Forest  0.99970      0.98
0  Logistic Regression  0.58790      0.50
3                  SVM  0.41215      0.50

✅ ЛУЧШАЯ МОДЕЛЬ:
   → Gradient Boosting (AUC = 1.0000, Accuracy = 1.0000)
   → Параметры: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 100}


ВЫВОДЫ ПО ЗАДАЧЕ КЛАССИФИКАЦИИ SI:

1. Задача: определить, обладает ли соединение высокой селективностью (SI > медианы).
   - Это означает, что оно эффективно и безопасно.

2. Все модели показали хорошее качество (AUC > 0.8), но немного хуже, чем для IC50/CC50.
   - Причина: SI — производная величина, и шум в IC50 и CC50 усиливается.

3. Лучшие результаты — у Random Forest и Gradient Boosting, так как они лучше работают с нелинейными комбинациями признаков.

4. Logistic Regression — хороша для интерпретации, но уступает в AUC.

5. Интересно: соединения с высоким SI — это золотая середина между активностью и безопасностью.
   - Их доля — около 50% (по медиане), но именно они перспективны для дальнейшей разработки.

6. Рекомендация:
   - Использовать ансамбли деревьев для предварительного отбора селективных соединений.
   - Можно комбинировать с порогом SI > 8 для более строгого скрининга.